In [2]:
from xgboost import XGBRegressor

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg'

from tqdm import tqdm

import matplotlib.pylab as plt
%matplotlib inline

import os
os.chdir('..')

In [3]:
transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else x)
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

for day_shift in tqdm(np.arange(-1, 2)):
    for month_shift in train_transactions.month_num.unique()[1:]:
#     for month_shift in np.arange(7):
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['neg_amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['neg_amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'neg_amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        
train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
train.drop(['pos_amount', 'n_transactions'], axis=1, inplace=True)
train.head()



100%|██████████| 3/3 [00:06<00:00,  1.92s/it]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,neg_amount_day_-1_1,neg_amount_day_-1_2,neg_amount_day_-1_3,neg_amount_day_-1_4,neg_amount_day_-1_5,neg_amount_day_-1_6,neg_amount_day_-1_7,neg_amount_day_-1_8,neg_amount_day_-1_9,neg_amount_day_-1_10,neg_amount_day_-1_11,neg_amount_day_-1_12,neg_amount_day_-1_13,neg_amount_day_-1_14,neg_amount_day_-1_15,neg_amount_day_0_1,neg_amount_day_0_2,neg_amount_day_0_3,neg_amount_day_0_4,neg_amount_day_0_5,neg_amount_day_0_6,neg_amount_day_0_7,neg_amount_day_0_8,neg_amount_day_0_9,neg_amount_day_0_10,neg_amount_day_0_11,neg_amount_day_0_12,neg_amount_day_0_13,neg_amount_day_0_14,neg_amount_day_0_15,neg_amount_day_1_1,neg_amount_day_1_2,neg_amount_day_1_3,neg_amount_day_1_4,neg_amount_day_1_5,neg_amount_day_1_6,neg_amount_day_1_7,neg_amount_day_1_8,neg_amount_day_1_9,neg_amount_day_1_10,neg_amount_day_1_11,neg_amount_day_1_12,neg_amount_day_1_13,neg_amount_day_1_14,neg_amount_day_1_15
0,4814,0,0,0,0,0,-11098744,0,2365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4814,1,0,1,0,1,-7881825,0,1697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4814,2,0,2,0,2,-6777480,0,1524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4814,3,0,3,0,3,-9277943,0,1937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4814,4,0,4,0,4,-9999757,0,1943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def rmsle(predicted, actual):
    assert(len(predicted) == len(actual))
    p = np.log(np.array(predicted) + 1)
    a = np.log(np.array(actual) + 1)
    return (((p - a)**2).sum() / len(predicted))**0.5

def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        clf.fit(train_sample[c], np.log1p(-train_sample[target_col_name]))
        predicted_volume = np.expm1(clf.predict(test_sample[c]))
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle(
            predicted_volume, -test_sample[target_col_name])
    return metric_by_shift


In [7]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression(n_jobs=-1)
metric_by_shift = eval_model(train, 'neg_amount', clf)
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 6/6 [00:04<00:00,  1.20it/s]

RMSLE with 90 days: 3.502293288680388
RMSLE with 75 days: 3.4970551323329233
RMSLE with 60 days: 3.4720487301653438
RMSLE with 45 days: 3.464594247687256
RMSLE with 30 days: 3.4540500704754122
RMSLE with 15 days: 3.457445866820557


In [13]:
transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else x)
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

for day_shift in tqdm(np.arange(-1, 2)):
#     for month_shift in train_transactions.month_num.unique()[1:]:
    for month_shift in np.arange(1, 7):
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['neg_amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['neg_amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'neg_amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
train.drop(['pos_amount', 'n_transactions'], axis=1, inplace=True)

train.head()

100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


,day,week_num,week_day,month_num,month_day,neg_amount,neg_amount_day_-1_1,neg_amount_day_-1_2,neg_amount_day_-1_3,neg_amount_day_-1_4,neg_amount_day_-1_5,neg_amount_day_-1_6,neg_amount_day_0_1,neg_amount_day_0_2,neg_amount_day_0_3,neg_amount_day_0_4,neg_amount_day_0_5,neg_amount_day_0_6,neg_amount_day_1_1,neg_amount_day_1_2,neg_amount_day_1_3,neg_amount_day_1_4,neg_amount_day_1_5,neg_amount_day_1_6,mcc_code_742,mcc_code_1711,mcc_code_1731,mcc_code_1799,mcc_code_2741,mcc_code_3000,mcc_code_3351,mcc_code_3501,mcc_code_4111,mcc_code_4112,mcc_code_4121,mcc_code_4131,mcc_code_4214,mcc_code_4215,mcc_code_4411,mcc_code_4511,mcc_code_4722,mcc_code_4784,mcc_code_4789,mcc_code_4812,mcc_code_4814,mcc_code_4816,mcc_code_4829,mcc_code_4899,mcc_code_4900,mcc_code_5013,mcc_code_5039,mcc_code_5044,mcc_code_5045,mcc_code_5047,mcc_code_5065,mcc_code_5072,mcc_code_5074,mcc_code_5085,mcc_code_5094,mcc_code_5099,mcc_code_5111,mcc_code_5122,mcc_code_5131,mcc_code_5137,mcc_code_5169,mcc_code_5172,mcc_code_5192,mcc_code_5193,mcc_code_5199,mcc_code_5200,mcc_code_5211,mcc_code_5231,mcc_code_5251,mcc_code_5261,mcc_code_5300,mcc_code_5309,mcc_code_5310,mcc_code_5311,mcc_code_5331,mcc_code_5399,mcc_code_5411,mcc_code_5422,mcc_code_5441,mcc_code_5451,mcc_code_5462,mcc_code_5499,mcc_code_5511,mcc_code_5532,mcc_code_5533,mcc_code_5541,mcc_code_5542,mcc_code_5571,mcc_code_5599,mcc_code_5611,mcc_code_5621,mcc_code_5631,mcc_code_5641,mcc_code_5651,mcc_code_5655,mcc_code_5661,...,mcc_code_5722,mcc_code_5732,mcc_code_5733,mcc_code_5734,mcc_code_5735,mcc_code_5811,mcc_code_5812,mcc_code_5813,mcc_code_5814,mcc_code_5816,mcc_code_5912,mcc_code_5921,mcc_code_5931,mcc_code_5940,mcc_code_5941,mcc_code_5942,mcc_code_5943,mcc_code_5944,mcc_code_5945,mcc_code_5946,mcc_code_5947,mcc_code_5948,mcc_code_5949,mcc_code_5950,mcc_code_5964,mcc_code_5965,mcc_code_5967,mcc_code_5968,mcc_code_5969,mcc_code_5970,mcc_code_5971,mcc_code_5976,mcc_code_5977,mcc_code_5983,mcc_code_5992,mcc_code_5993,mcc_code_5994,mcc_code_5995,mcc_code_5999,mcc_code_6010,mcc_code_6011,mcc_code_6012,mcc_code_6051,mcc_code_6211,mcc_code_6300,mcc_code_6513,mcc_code_6536,mcc_code_7011,mcc_code_7210,mcc_code_7216,mcc_code_7221,mcc_code_7230,mcc_code_7273,mcc_code_7278,mcc_code_7298,mcc_code_7299,mcc_code_7311,mcc_code_7338,mcc_code_7372,mcc_code_7375,mcc_code_7395,mcc_code_7399,mcc_code_7512,mcc_code_7523,mcc_code_7531,mcc_code_7538,mcc_code_7542,mcc_code_7629,mcc_code_7699,mcc_code_7829,mcc_code_7832,mcc_code_7841,mcc_code_7922,mcc_code_7932,mcc_code_7933,mcc_code_7991,mcc_code_7993,mcc_code_7994,mcc_code_7995,mcc_code_7996,mcc_code_7997,mcc_code_7999,mcc_code_8011,mcc_code_8021,mcc_code_8043,mcc_code_8062,mcc_code_8071,mcc_code_8099,mcc_code_8220,mcc_code_8244,mcc_code_8299,mcc_code_8398,mcc_code_8641,mcc_code_8699,mcc_code_8999,mcc_code_9211,mcc_code_9222,mcc_code_9311,mcc_code_9399,mcc_code_9402
0,0,0,0,0,0,-11098744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,-7881825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,2,-6777480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [14]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression(n_jobs=-1)
metric_by_shift = eval_model(train, 'neg_amount', clf)
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 6/6 [00:03<00:00,  1.53it/s]

RMSLE with 90 days: 3.501818856165513
RMSLE with 75 days: 3.4994169317849897
RMSLE with 60 days: 3.4734172478772685
RMSLE with 45 days: 3.4715412526744887
RMSLE with 30 days: 3.457872008132199
RMSLE with 15 days: 3.4672710645330493


In [17]:
%time transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else x)
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

for day_shift in tqdm(np.arange(-1, 2)):
    for month_shift in train_transactions.month_num.unique()[1:]:
#     for month_shift in np.arange(1, 10):
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['neg_amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['neg_amount'] + 1)
        train_shift['n_transact_day_{}_{}'.format(day_shift, month_shift)] = train_shift['n_transactions']
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'neg_amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
train.drop(['pos_amount', 'n_transactions'], axis=1, inplace=True)

from sklearn.linear_model import LinearRegression

clf = LinearRegression(n_jobs=-1)
metric_by_shift = eval_model(train, 'neg_amount', clf)
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

CPU times: user 9.87 s, sys: 480 ms, total: 10.4 s
Wall time: 10.4 s


100%|██████████| 6/6 [00:04<00:00,  1.19it/s]

RMSLE with 90 days: 3.502293288680388
RMSLE with 75 days: 3.4970551323329233
RMSLE with 60 days: 3.4720487301653438
RMSLE with 45 days: 3.464594247687256
RMSLE with 30 days: 3.4540500704754122
RMSLE with 15 days: 3.457445866820557


In [19]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.05, max_depth=8, n_estimators=250, nthread=-1, reg_alpha=1, reg_lambda=1)
metric_by_shift = eval_model(train, 'neg_amount', clf, day_shifts=[60, 30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 2/2 [02:05<00:00, 59.16s/it]

RMSLE with 60 days: 3.4797221178463236
RMSLE with 30 days: 3.461679758335893


In [31]:
%time transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else x)
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

for day_shift in tqdm(np.arange(-1, 2)):
    for month_shift in train_transactions.month_num.unique()[1:]:
#     for month_shift in np.arange(1, 10):
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['neg_amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['neg_amount'] + 1)
#         train_shift['n_transact_day_{}_{}'.format(day_shift, month_shift)] = train_shift['n_transactions']
        train_shift['pos_amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(train_shift['pos_amount'] + 1)
        train_shift = train_shift[[
            'month_num', 'month_day', 'mcc_code', 
            'neg_amount_day_{0}_{1}'.format(day_shift, month_shift),
            'pos_amount_day_{0}_{1}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
train.drop(['pos_amount', 'n_transactions'], axis=1, inplace=True)

from sklearn.linear_model import LinearRegression

clf = LinearRegression(n_jobs=-1)
metric_by_shift = eval_model(train, 'neg_amount', clf)
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

CPU times: user 9.53 s, sys: 464 ms, total: 9.99 s
Wall time: 9.99 s


100%|██████████| 6/6 [00:06<00:00,  1.05s/it]

RMSLE with 90 days: 3.5033065435945825
RMSLE with 75 days: 3.4981579005655616
RMSLE with 60 days: 3.4735337019406582
RMSLE with 45 days: 3.465823930203214
RMSLE with 30 days: 3.4568579929827563
RMSLE with 15 days: 3.4592218788675


In [32]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.05, max_depth=8, n_estimators=250, nthread=-1, reg_alpha=1, reg_lambda=1)
metric_by_shift = eval_model(train, 'neg_amount', clf, day_shifts=[60, 30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 2/2 [02:02<00:00, 60.65s/it]

RMSLE with 60 days: 3.481049761516421
RMSLE with 30 days: 3.4570654050981475


In [33]:
clf = XGBRegressor(learning_rate=0.05, max_depth=10, n_estimators=450, nthread=-1, reg_alpha=1, reg_lambda=1)
xgb_test = test_transactions.copy()

c = train.columns.difference(['neg_amount'])

%time clf.fit(train[c], np.log1p(-train['neg_amount']))
xgb_test['volume'] = np.expm1(clf.predict(test[c]))
xgb_test['id'] = xgb_test[['mcc_code', 'day']].apply(lambda x: '-'.join(map(str, x)), axis=1)
xgb_test[['id', 'volume']].to_csv('xgb_baseline_with_numtr_pos_amount.csv', index=False)
xgb_test[['id', 'volume']].head()

CPU times: user 41min 21s, sys: 31.1 s, total: 41min 52s
Wall time: 2min 38s


,id,volume
0,4814-457,9157594.0
1,4814-458,10869541.0
2,4814-459,10680694.0
3,4814-460,10207426.0
4,4814-461,10861521.0


In [39]:
xgb_test.volume = xgb_test.volume.apply(lambda x: 0 if x<0 else x)
xgb_test[['id', 'volume']].to_csv('xgb_baseline_with_numtr_pos_amount.csv', index=False)
xgb_test.head()

,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount_day_-1_1,pos_amount_day_-1_1,neg_amount_day_-1_2,pos_amount_day_-1_2,neg_amount_day_-1_3,pos_amount_day_-1_3,neg_amount_day_-1_4,pos_amount_day_-1_4,neg_amount_day_-1_5,pos_amount_day_-1_5,neg_amount_day_-1_6,pos_amount_day_-1_6,neg_amount_day_-1_7,pos_amount_day_-1_7,neg_amount_day_-1_8,pos_amount_day_-1_8,neg_amount_day_-1_9,pos_amount_day_-1_9,neg_amount_day_-1_10,pos_amount_day_-1_10,neg_amount_day_-1_11,pos_amount_day_-1_11,neg_amount_day_-1_12,pos_amount_day_-1_12,neg_amount_day_-1_13,pos_amount_day_-1_13,neg_amount_day_-1_14,pos_amount_day_-1_14,neg_amount_day_-1_15,pos_amount_day_-1_15,neg_amount_day_0_1,pos_amount_day_0_1,neg_amount_day_0_2,pos_amount_day_0_2,neg_amount_day_0_3,pos_amount_day_0_3,neg_amount_day_0_4,pos_amount_day_0_4,neg_amount_day_0_5,pos_amount_day_0_5,neg_amount_day_0_6,pos_amount_day_0_6,neg_amount_day_0_7,pos_amount_day_0_7,neg_amount_day_0_8,pos_amount_day_0_8,neg_amount_day_0_9,pos_amount_day_0_9,neg_amount_day_0_10,pos_amount_day_0_10,neg_amount_day_0_11,pos_amount_day_0_11,neg_amount_day_0_12,pos_amount_day_0_12,neg_amount_day_0_13,pos_amount_day_0_13,neg_amount_day_0_14,pos_amount_day_0_14,neg_amount_day_0_15,pos_amount_day_0_15,neg_amount_day_1_1,pos_amount_day_1_1,neg_amount_day_1_2,pos_amount_day_1_2,neg_amount_day_1_3,pos_amount_day_1_3,neg_amount_day_1_4,pos_amount_day_1_4,neg_amount_day_1_5,pos_amount_day_1_5,neg_amount_day_1_6,pos_amount_day_1_6,neg_amount_day_1_7,pos_amount_day_1_7,neg_amount_day_1_8,pos_amount_day_1_8,neg_amount_day_1_9,pos_amount_day_1_9,neg_amount_day_1_10,pos_amount_day_1_10,neg_amount_day_1_11,pos_amount_day_1_11,neg_amount_day_1_12,pos_amount_day_1_12,neg_amount_day_1_13,pos_amount_day_1_13,neg_amount_day_1_14,pos_amount_day_1_14,neg_amount_day_1_15,pos_amount_day_1_15,volume,id
0,4814,457,65,2,15,7,16.000073,0.0,16.239146,0.0,16.396831,0.000000,15.931877,0.0,16.212155,0.0,16.220646,0.000000,16.121187,11.605222,15.975065,0.000000,16.122443,0.0,15.935854,0.0,15.728413,0.000000,16.105708,0.0,16.155763,0.0,16.057572,0.0,15.849065,0.0,16.394115,0.0,16.403887,0.0,16.230284,0.0,16.115614,7.023759,16.141662,0.0,16.119394,0.000000,15.847769,0.000000,16.263352,0.0,16.108720,0.0,15.775248,0.0,16.039852,0.000000,16.026004,0.0,16.161532,0.0,15.685140,0.0,16.022030,9.222763,16.513388,0.0,16.568458,0.0,16.186382,0.0,16.337125,0.000000,16.201899,0.0,15.895226,0.0,15.972526,0.000000,16.170612,12.307537,16.114276,0.0,15.794399,0.0,16.125836,9.077951,16.102922,0.0,16.167790,0.0,15.776396,0.0,16.072431,0.000000,9157594.0,4814-457
1,4814,458,65,3,15,8,15.909834,0.0,16.309198,0.0,16.124614,0.000000,16.217592,0.0,16.188250,0.0,16.247044,0.000000,16.068822,0.000000,16.297282,0.000000,16.120781,0.0,15.916387,0.0,16.185141,0.000000,15.929218,0.0,16.113109,0.0,16.048617,0.0,15.718409,0.0,16.000073,0.0,16.239146,0.0,16.396831,0.0,15.931877,0.000000,16.212155,0.0,16.220646,0.000000,16.121187,11.605222,15.975065,0.0,16.122443,0.0,15.935854,0.0,15.728413,0.000000,16.105708,0.0,16.155763,0.0,16.057572,0.0,15.849065,0.000000,16.394115,0.0,16.403887,0.0,16.230284,0.0,16.115614,7.023759,16.141662,0.0,16.119394,0.0,15.847769,0.000000,16.263352,0.000000,16.108720,0.0,15.775248,0.0,16.039852,0.000000,16.026004,0.0,16.161532,0.0,15.685140,0.0,16.022030,9.222763,10869541.0,4814-458
2,4814,459,65,4,15,9,16.217647,0.0,15.970313,0.0,16.174195,0.000000,16.118953,0.0,15.884278,0.0,16.274036,0.000000,16.170330,9.355911,15.910726,0.000000,15.956240,0.0,15.844803,0.0,16.164770,0.000000,15.779407,0.0,16.185010,0.0,16.087859,0.0,16.157151,0.0,15.909834,0.0,16.309198,0.0,16.124614,0.0,16.217592,0.000000,16.188250,0.0,16.247044,0.000000,16.068822,0.000000,16.297282,0.0,16.120781,0.0,15.916387,0.0,16.185141,0.000000,15.929218,0.0,16.113109,0.0,16.048617,0.0,15.718409,0.000000,16.000073,0.0,16.239146,0.0,16.396831,0.0,15.931877,0.000000,16.212155,0.0,16.220646,0.0,16.121187,11.605222,15.975065,0.000000,16.122443,0.0,15.935854,0.0,15.728413,0.000000,16

In [ ]:
def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        clf.fit(train_sample[c], np.log1p(-train_sample[target_col_name]))
        predicted_volume = np.expm1(clf.predict(test_sample[c]))
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle(
            predicted_volume, -test_sample[target_col_name])
    return metric_by_shift

def get_prediction(train, test, target_col_name, clf):
    